In [ ]:
# 安装必要的依赖
# 如果已经安装可以跳过此步骤
# %pip install openai -q


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 如果导入失败，请先运行上面的安装单元格，然后重启内核（Kernel -> Restart Kernel）
from openai import OpenAI

# python 实例化 不需要new对象，，没有{}
client = OpenAI(
    api_key='sk-676db137e5804e5db22375c19a69d580',
    base_url='https://api.deepseek.com/v1'
)

In [3]:
# 轻量级数据库
import sqlite3

In [4]:
# 打开链接
conn=sqlite3.connect("test3.db")
# 数据库操作的句柄
cursor=conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS employees(
    id INTEGER PRIMARY KEY,
    name TEXT,
    department TEXT,
    salary INTEGER
)
""")

In [5]:
# list 
sample_data = [ 
    # 元祖类型 有顺序的  tuple 
    (6, "黄佳", "销售", 50000), 
    (7, "宁宁", "工程", 75000), 
    (8, "谦谦", "销售", 60000), 
    (9, "悦悦", "工程", 80000), 
    (10, "黄仁勋", "市场", 55000),
    (11, "杜经理", "工程", 80000)
]
# 插入sql - 使用 INSERT OR IGNORE 避免重复插入时的错误
# 如果数据已存在，会静默跳过，不会报错
cursor.executemany(
    "INSERT OR IGNORE INTO employees VALUES(?,?,?,?)",
    sample_data
)
# 提交sql事务
conn.commit()
print(f"成功插入/跳过 {len(sample_data)} 条数据")

成功插入/跳过 6 条数据


In [6]:
# text2sql 给llm什么？ context
# llm会sql gemini 博士级别
# 表结构？ 作为上下文给prompt schema
schema = cursor.execute("PRAGMA table_info(employees)").fetchall()# 执行 PRAGMA table_info 语句，获取表 employees 的列信息
# fetchall 方法用于获取执行 SQL 语句后返回的所有行数据。
# 在这里，fetchall 方法用于获取表 employees 的列信息，每个元素都是一个元组，包含列的索引、列名、列的数据类型和是否为主键等信息。
print(schema)
schema_str = "CREATE TABLE EMPLOYEES (\n " + " \n ".join([f"{col[1]} {col[2]}" for col in schema]) + "\n)"
# 这句代码 是将表 employees 的列信息格式化为一个 SQL 语句字符串。
# 具体来说，它做了以下几件事：
# 1. 从 schema 中提取列名和数据类型，格式化为 "列名 数据类型" 的字符串。
# 2. 使用 "\n".join 方法将这些字符串连接起来，每个字符串之间用换行符分隔。
# 3. 在字符串的开头添加 "CREATE TABLE EMPLOYEES (\n"，在结尾添加 "\n)"，形成一个完整的 SQL 语句字符串。
# 为什么col[1] 是列名，col[2] 是数据类型？
# 因为 PRAGMA table_info 语句返回的每个元组中，列名是第二个元素，数据类型是第三个元素。
# 你可以在 PRAGMA table_info 语句的文档中找到更多关于这个的信息。

print(schema_str)

[(0, 'id', 'INTEGER', 0, None, 1), (1, 'name', 'TEXT', 0, None, 0), (2, 'department', 'TEXT', 0, None, 0), (3, 'salary', 'INTEGER', 0, None, 0)]
CREATE TABLE EMPLOYEES (
 id INTEGER 
 name TEXT 
 department TEXT 
 salary INTEGER
)


In [7]:
def ask_deepseek(query, schema):
    prompt = f"""
    这是一个数据库的Schema:
    {schema}
    根据这个Schema,你能输出一个SQL查询来回答以下问题吗？
    只输出SQL查询,不要输出任何内容。
    问题：{query}
    """
    # print(prompt)
    response = client.chat.completions.create(
        model="deepseek-reasoner",
        max_tokens=2048,
        messages=[{
            "role": "user",
            "content": prompt
        }]
    )
    return response.choices[0].message
    
# ask_deepseek("开发部部门员工的姓名和工资是多少？", schema)
question = "开发部部门员工的姓名和工资是多少？"
sql = ask_deepseek(question, schema_str)
print("生成的sql查询：")
print(sql)

生成的sql查询：
ChatCompletionMessage(content="```sql\nSELECT name, salary FROM EMPLOYEES WHERE department = '开发部';\n```", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning_content='我们被问到："开发部部门员工的姓名和工资是多少？" 注意Schema是：\nCREATE TABLE EMPLOYEES (\n id INTEGER \n name TEXT \n department TEXT \n salary INTEGER\n)\n问题：开发部部门员工的姓名和工资是多少？所以我们需要选择name和salary，其中department = \'开发部\'。注意：在Schema中，department是TEXT类型，所以我们应该在查询中使用字符串字面量。在SQL中，字符串通常用单引号括起来。所以查询是：\nSELECT name, salary FROM EMPLOYEES WHERE department = \'开发部\';\n输出应该只输出SQL查询，不要其他内容。')
